# Amazon 상품 Review 데이터 가져오기
* 반복 코드 수정 (add)
* 제품을 검색하여 상품 review받기 (add)
* wordcloud 추가 (add)
* Selenium과 BeautifulSoup을 이용

In [168]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import os, warnings 
import re
import time
warnings.filterwarnings(action='ignore') #warning message 무시하고 숨기기
from wordcloud import WordCloud
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

### 1) Amazon 사이트 열기

In [169]:
driver = webdriver.Chrome('chromedriver')
url='https://www.amazon.com/'
driver.get(url)

### 2) 키워드 입력후 찾기

In [170]:
#키워드 입력란
sel_search=driver.find_element_by_xpath('//*[@id="twotabsearchtextbox"]')
#검색 버튼
sel_btn= driver.find_element_by_xpath('//*[@id="nav-search-submit-button"]')

sel_search.clear() #값을 지움
sel_search.send_keys('dump truck toy') #입력

sel_btn.click() #클릭

### 3) 첫번째 제품 클릭
* sponsored를 제외하고 첫번째 제품 

In [171]:
searchResult = driver.page_source
soup = BeautifulSoup(searchResult,'lxml')

itemList = driver.find_elements_by_css_selector("div[class='sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col sg-col-4-of-20']")
firstUrl = itemList[0].find_element_by_css_selector("span[class='a-size-base-plus a-color-base a-text-normal']")

firstUrl.click()

### 4) 리뷰 클릭하고 전체 리뷰 보기

In [172]:
reviewSee = driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]')
reviewSee.click()

reviewAll = driver.find_element_by_xpath('//*[@id="cr-pagination-footer-0"]/a')
reviewAll.click()

### 5) review 정보 가지고 오기
* review 내용 , 작성자, 평점, 작성날짜, 작성지역, 유용함 정도
* 10개 페이지 내용 가지고 오기

In [173]:
review=[]
reviewer =[]
review_rate=[]
review_date=[]
review_area=[]
review_helpful=[]

#여러 페이지 가지고 오기
for i in range(10):
    page = driver.page_source
    soup = BeautifulSoup(page,'lxml')
    
    #한 페이지에서 데이터 가지고 오기
    review_all = soup.find("div",id="cm_cr-review_list").find_all("div",class_="a-section celwidget")

    for j in review_all:
        review.append(j.find("span",attrs={"data-hook":"review-body"}).text.strip())        
        reviewer.append(j.find("span",class_="a-profile-name").text)       
        review_rate.append(j.find("span",class_="a-icon-alt").text.split(' ')[0])
        date_area=j.find("span",attrs={"data-hook":"review-date"}).text.lstrip('Reviewed in the').split(' on ')
        review_date.append(date_area[-1])      
        review_area.append(date_area[0])
      
        helpful_count= j.find("span",attrs={"data-hook":"helpful-vote-statement"})
        if helpful_count is not None:
            helpful_count = j.find("span",attrs={"data-hook":"helpful-vote-statement"}).text.split(' ')[0]
            if helpful_count == 'One':
                helpful_count = 1
        else:
            helpful_count =0
        review_helpful.append(helpful_count) 
        
    nextPage=driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a')
    nextPage.click()    
    time.sleep(1)
    
print(reviewer, review_rate,review_date, review_area, review_helpful )


['Lil Bit of Sassiness', 'Janos', 'Billy  Rhoton', 'Kate', 'K', 'D.R.', 'Rachel', 'Chase kacsor', 'Kel', 'Tammy2566', 'Lil Bit of Sassiness', 'Janos', 'Billy  Rhoton', 'Kate', 'K', 'D.R.', 'Rachel', 'Chase kacsor', 'Kel', 'Tammy2566', 'Lil Bit of Sassiness', 'Janos', 'Billy  Rhoton', 'Kate', 'K', 'D.R.', 'Rachel', 'Chase kacsor', 'Kel', 'Tammy2566', 'Lil Bit of Sassiness', 'Janos', 'Billy  Rhoton', 'Kate', 'K', 'D.R.', 'Rachel', 'Chase kacsor', 'Kel', 'Tammy2566', 'Lil Bit of Sassiness', 'Janos', 'Billy  Rhoton', 'Kate', 'K', 'D.R.', 'Rachel', 'Chase kacsor', 'Kel', 'Tammy2566', 'Lil Bit of Sassiness', 'Janos', 'Billy  Rhoton', 'Kate', 'K', 'D.R.', 'Rachel', 'Chase kacsor', 'Kel', 'Tammy2566', 'Lil Bit of Sassiness', 'Janos', 'Billy  Rhoton', 'Kate', 'K', 'D.R.', 'Rachel', 'Chase kacsor', 'Kel', 'Tammy2566', 'Lil Bit of Sassiness', 'Janos', 'Billy  Rhoton', 'Kate', 'K', 'D.R.', 'Rachel', 'Chase kacsor', 'Kel', 'Tammy2566', 'Lil Bit of Sassiness', 'Janos', 'Billy  Rhoton', 'Kate', 'K', 

### 6) 데이터 프레임 만들어 csv파일 생성

In [174]:
data ={'review': review, 
       'reviewer':reviewer, 
       'rate':review_rate ,
       'review_date': review_date,
       'area':review_area,
       'helpful_count':review_helpful}

data = pd.DataFrame(data)
display(data)
data.to_csv('amazon_review.csv',index=False, encoding='utf-8-sig')

,review,reviewer,rate,review_date,area,helpful_count
0,This truck is perfect for our grandson! He lov...,Lil Bit of Sassiness,5.0,"October 30, 2019",United States,20
1,The title of the review pretty much sums this ...,Janos,5.0,"May 4, 2020",United States,11
2,I was kinda skeptical due to the price but I a...,Billy Rhoton,5.0,"December 14, 2019",United States,13
3,Our little one really enjoys this big dump tru...,Kate,5.0,"January 10, 2020",United States,5
4,my son loves to take this outside in the yard....,K,5.0,"February 29, 2020",United States,6
...,...,...,...,...,...,...
95,"The CAT Bulldozer is good looking, but the tre...",D.R.,3.0,"August 24, 2020",United States,2
96,"Was expecting it to be larger, but I didnt che...",Rachel,3.0,"December 9, 2019",United States,3
97,Good dump truck for kids. My little one loves ...,Chase kacsor,5.0,"September 7, 2020",United States,4
98,My 2 year old son loves this toy and plays wit...,Kel,5.0,"January 19, 2021",United States,0


PermissionError: [Errno 13] Permission denied: 'amazon_review.csv'

### 7) 워드클라우드 나타내기

In [ ]:
print(type(review)) #type이 list라서 String 문자열로 변환해줌

wordcloud = WordCloud(max_font_size=200,background_color='white', width=800, height=800).generate(''.join(review))

plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.tight_layout(pad=0)
plt.axis('off')
plt.show()
